In [ ]:
import numpy as np

image_dateset = '../datasets/unlabelled_train_data_images.npy'
Xtrain_img = np.load(image_dateset)


In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def preprocess_images(
    X, 
    normalize=True, 
    standardize=False, 
    split=False, 
    test_size=0.2, 
    random_state=42
):
    """
    Preprocess image data 
    
    Parameters:
    -----------
    X : np.ndarray
        Image data of shape (n_samples, 1, height, width)
    normalize : bool, default=True
        If True, scales pixel values to [0, 1].
    standardize : bool, default=False
        If True, applies zero mean/unit variance scaling after normalization.
    split : bool, default=False
        If True, returns a train/validation split.
    test_size : float, default=0.2
        Fraction of data to use as validation set if split is True.
    random_state : int, default=42
        Random state for reproducibility.
        
    Returns:
    --------
    X_processed : np.ndarray or tuple
        Preprocessed data, or (X_train, X_test) if split is True.
    """
    # Flatten images
    X_flat = X.reshape(X.shape[0], -1)
    
    # Normalize pixel values
    if normalize:
        X_flat = X_flat.astype('float32') / 255.0
    
    # Standardize if required
    if standardize:
        scaler = StandardScaler()
        X_flat = scaler.fit_transform(X_flat)
    
    # Train/test split if required
    if split:
        X_train, X_test = train_test_split(X_flat, test_size=test_size, random_state=random_state)
        return X_train, X_test
    
    return X_flat


In [4]:
X_flat = preprocess_images(Xtrain_img, standardize=True)
# X_train, X_val = preprocess_images(Xtrain_img, standardize=True train_val_split=True)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

def visualize_digit_clusters(X_processed, n_components=2, perplexity=30):
    """
    Visualize clusters in the processed image data using t-SNE.
    
    Parameters:
    -----------
    X_processed : np.ndarray
        Preprocessed image data of shape (n_samples, n_features)
    n_components : int, default=2
        Number of dimensions for visualization
    perplexity : int, default=30
        t-SNE perplexity parameter
        
    Returns:
    --------
    embeddings : np.ndarray
        t-SNE embeddings of shape (n_samples, n_components)
    """
    # Apply t-SNE for visualization
    print("Applying t-SNE dimensionality reduction...")
    tsne = TSNE(n_components=n_components, perplexity=perplexity, random_state=42)
    embeddings = tsne.fit_transform(X_processed)
    
    # Plot the embeddings
    plt.figure(figsize=(10, 8))
    plt.scatter(embeddings[:, 0], embeddings[:, 1], alpha=0.5, s=5)
    plt.title('t-SNE visualization of MNIST digits')
    plt.xlabel('t-SNE dimension 1')
    plt.ylabel('t-SNE dimension 2')
    plt.colorbar()
    plt.show()
    
    return embeddings

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def cluster_digits(X_processed, n_clusters=10, random_state=42):
    """
    Cluster the processed image data using K-means.
    
    Parameters:
    -----------
    X_processed : np.ndarray
        Preprocessed image data of shape (n_samples, n_features)
    n_clusters : int, default=10
        Number of clusters (10 for MNIST digits)
    random_state : int, default=42
        Random state for reproducibility
        
    Returns:
    --------
    kmeans : KMeans
        Fitted K-means model
    cluster_labels : np.ndarray
        Predicted cluster labels
    """
    # Apply K-means clustering
    print(f"Applying K-means clustering with {n_clusters} clusters...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    cluster_labels = kmeans.fit_predict(X_processed)
    
    # Evaluate clustering quality
    silhouette_avg = silhouette_score(X_processed, cluster_labels)
    print(f"Silhouette Score: {silhouette_avg:.4f}")
    
    return kmeans, cluster_labels

def visualize_cluster_centers(kmeans, image_shape=(28, 28)):
    """
    Visualize the cluster centers as images.
    
    Parameters:
    -----------
    kmeans : KMeans
        Fitted K-means model
    image_shape : tuple, default=(28, 28)
        Shape of the original images
    """
    # Reshape cluster centers to original image dimensions
    centers = kmeans.cluster_centers_.reshape(-1, *image_shape)
    
    # Plot cluster centers
    fig, axes = plt.subplots(2, 5, figsize=(12, 5))
    axes = axes.flatten()
    
    for i, center in enumerate(centers):
        axes[i].imshow(center, cmap='gray')
        axes[i].set_title(f'Cluster {i}')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def train_autoencoder(X_processed, encoding_dim=32, hidden_layer_sizes=(128, 64), 
                      random_state=42, test_size=0.2):
    """
    Train an autoencoder using MLPRegressor for feature learning.
    
    Parameters:
    -----------
    X_processed : np.ndarray
        Preprocessed image data of shape (n_samples, n_features)
    encoding_dim : int, default=32
        Dimension of the encoded representation
    hidden_layer_sizes : tuple, default=(128, 64)
        Sizes of hidden layers before the bottleneck
    random_state : int, default=42
        Random state for reproducibility
    test_size : float, default=0.2
        Fraction of data to use as validation
        
    Returns:
    --------
    encoder : MLPRegressor
        Fitted encoder model
    encoded_features : np.ndarray
        Encoded features for all samples
    """
    # Split data for validation
    X_train, X_val = train_test_split(X_processed, test_size=test_size, 
                                     random_state=random_state)
    
    # Define the full autoencoder architecture
    full_architecture = hidden_layer_sizes + (encoding_dim,) + hidden_layer_sizes[::-1]
    
    # Train the autoencoder
    print("Training autoencoder for unsupervised pre-training...")
    autoencoder = MLPRegressor(
        hidden_layer_sizes=full_architecture,
        activation='relu',
        solver='adam',
        max_iter=200,
        verbose=True,
        random_state=random_state
    )
    
    # Train to reconstruct the input
    autoencoder.fit(X_train, X_train)
    
    # Evaluate reconstruction performance
    X_val_pred = autoencoder.predict(X_val)
    mse = mean_squared_error(X_val, X_val_pred)
    print(f"Reconstruction MSE: {mse:.6f}")
    
    # Create a function to get encoded features
    def get_encoded_features(X):
        # This is a simplified approach - in practice with sklearn's MLPRegressor,
        # you would need to implement a custom method to extract intermediate layer outputs
        # Here we're simulating the encoding process
        encoded = X  # Placeholder for actual encoding
        return encoded
    
    encoded_features = get_encoded_features(X_processed)
    
    return autoencoder, encoded_features

def analyze_clusters(embeddings, cluster_labels, n_clusters=10):
    """
    Analyze the quality of clusters and visualize them.
    
    Parameters:
    -----------
    embeddings : np.ndarray
        t-SNE embeddings of shape (n_samples, 2)
    cluster_labels : np.ndarray
        Predicted cluster labels
    n_clusters : int, default=10
        Number of clusters
    """
    # Visualize clusters in the t-SNE space
    plt.figure(figsize=(10, 8))
    for cluster in range(n_clusters):
        # Select points in this cluster
        cluster_points = embeddings[cluster_labels == cluster]
        plt.scatter(
            cluster_points[:, 0], 
            cluster_points[:, 1], 
            label=f'Cluster {cluster}',
            alpha=0.6,
            s=5
        )
    
    plt.title('t-SNE visualization of MNIST digit clusters')
    plt.xlabel('t-SNE dimension 1')
    plt.ylabel('t-SNE dimension 2')
    plt.legend(markerscale=2)
    plt.show()
    
    # Calculate cluster sizes
    cluster_sizes = [np.sum(cluster_labels == i) for i in range(n_clusters)]
    
    # Plot cluster distribution
    plt.figure(figsize=(10, 6))
    plt.bar(range(n_clusters), cluster_sizes)
    plt.xlabel('Cluster')
    plt.ylabel('Number of samples')
    plt.title('Distribution of samples across clusters')
    plt.xticks(range(n_clusters))
    plt.show()

def process_unlabelled_mnist(X_processed):
    """
    Complete pipeline for processing unlabelled MNIST data.
    
    Parameters:
    -----------
    X_processed : np.ndarray
        Preprocessed image data of shape (n_samples, n_features)
    """
    # 1. Apply t-SNE for visualization
    embeddings = visualize_digit_clusters(X_processed)
    
    # 2. Cluster the digits
    kmeans, cluster_labels = cluster_digits(X_processed)
    
    # 3. Visualize cluster centers
    visualize_cluster_centers(kmeans)
    
    # 4. Train autoencoder for feature learning
    autoencoder, encoded_features = train_autoencoder(X_processed)
    
    # 5. Analyze clusters
    analyze_clusters(embeddings, cluster_labels)


Silhouette Score: 0.01


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_flat)

plt.scatter(X_tsne[:,0], X_tsne[:,1], c=cluster_labels, cmap='tab10', alpha=0.5)
plt.colorbar()


In [ ]:
# Basic analysis
# Shape analysis
print(f"Dataset shape: {Xtrain_img.shape}")  
print(f"Data type: {Xtrain_img.dtype}")       
print(f"Pixel range: {Xtrain_img.min()}–{Xtrain_img.max()}")  

In [ ]:
# Flatten images for pixel statistics i.e. reshape to 2D
flattened = Xtrain_img.reshape(Xtrain_img.shape[0], -1)
print(f"Flatted shape: {flattened.shape}")

# Per-image statistics
mean_per_image = flattened.mean(axis=1)
std_per_image = flattened.std(axis=1)

# Global statistics
global_mean = flattened.mean()
global_std = flattened.std()

print(f"flattened img mean: {global_mean:.2f}")
print(f"flattened img std: {global_std:.2f}")


In [ ]:

import matplotlib.pyplot as plt
# Display sample images
fig, axes = plt.subplots(3, 3, figsize=(8,8))
for i, ax in enumerate(axes.flat):
    ax.imshow(Xtrain_img[i][0], cmap='gray')  # Squeeze channel dimension
    ax.axis('off')
plt.tight_layout()
plt.show()

# Pixel intensity distribution
plt.hist(flattened.ravel(), bins=50)
plt.title("Pixel Intensity Distribution")
plt.xlabel("Pixel Value")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# Find images with extreme brightness
brightest_idx = np.argmax(mean_per_image)
darkest_idx = np.argmin(mean_per_image)

print(f"Brightest image index: {brightest_idx} (mean: {mean_per_image[brightest_idx]:.1f})")
print(f"Darkest image index: {darkest_idx} (mean: {mean_per_image[darkest_idx]:.1f})")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

def apply_standard_scaler(X_train=None, X_test=None):

    X_train_scaled = None
    X_test_scaled = None

    scaler = StandardScaler()
    if X_train is not None:
        X_train_scaled = scaler.fit_transform(X_train)
    
    if X_test is not None:
        X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

def apply_pca(X_train_scaled, X_test_scaled=None, variance_threshold=0.95, loading_threshold=0.3, 
              plot_heatmap=True, plot_bar=True, plot_variance=True):
    
    data = pd.DataFrame(X_train_scaled)
    feature_names = data.columns
    data_scaled = data.values

    max_components = min(data_scaled.shape[0], data_scaled.shape[1])
    pca = PCA(n_components=max_components)
    pca.fit(data_scaled)
    
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.where(cumulative_variance >= variance_threshold)[0][0] + 1
    total_variance = cumulative_variance[n_components - 1]
    
    pca = PCA(n_components=n_components)
    pca.fit(data_scaled)
    
    loadings_df = pd.DataFrame(
        pca.components_.T,
        index=feature_names,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )

    ## Simple way::
    # pca = PCA()
    # pca.fit(data_scaled)
    # cumsum = np.cumsum(pca.explained_variance_ratio_)
    # d = np.argmax(cumsum >= variance_threshold) + 1
    # n_components = d
    ## OR
    # pca = PCA(n_components=0.95)
    # X_reduced = pca.fit_transform(data_scaled)
    
    suggested_names = {}
    print(f"\nSelected {n_components} components to explain {total_variance*100:.2f}% of variance")
    for i, pc in enumerate(loadings_df.columns):
        dominant_feature = loadings_df[pc].abs().idxmax()
        max_loading = loadings_df[pc][dominant_feature]
        suggested_names[pc] = f"{dominant_feature}_Dominant (Loading: {max_loading:.3f})"
        
        pc_loadings = loadings_df[pc]
        dominant_cols = pc_loadings[np.abs(pc_loadings) > loading_threshold]
        if dominant_cols.empty:
            dominant_cols = pc_loadings[[pc_loadings.abs().idxmax()]]
        dominant_cols = dominant_cols.sort_values(key=abs, ascending=False)
        
        print(f"\nDominant Columns for {pc}:")
        for feature, loading in dominant_cols.items():
            print(f"- {feature}: Loading = {loading:.3f}, Absolute = {abs(loading):.3f}")
        print(f"PC{i+1}: Explains {pca.explained_variance_ratio_[i]*100:.2f}% of variance")
    
    if plot_heatmap:
        plt.figure(figsize=(12, 8))
        sns.heatmap(loadings_df, cmap='coolwarm', center=0, annot=True, fmt='.2f')
        plt.title('PCA Loadings: Feature Contributions to Principal Components')
        plt.xlabel('Principal Components')
        plt.ylabel('Features')
        plt.tight_layout()
        plt.show()
    
    if plot_bar:
        for pc in loadings_df.columns:
            plt.figure(figsize=(10, 6))
            loadings_df[pc].sort_values(ascending=False).plot(kind='bar')
            plt.title(f'Feature Loadings for {pc}')
            plt.xlabel('Features')
            plt.ylabel('Loading Value')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()

    if plot_variance:
        max_components = len(pca.explained_variance_ratio_)
        plot_variance_charts(pca.explained_variance_ratio_, cumulative_variance, max_components)
    
    print("\nSuggested Names for Principal Components:")
    for pc, name in suggested_names.items():
        print(f"{pc}: {name}")
    
    X_train_pca = pca.transform(X_train_scaled)

    X_test_pca = None
    if X_test_scaled is not None:
        X_test_pca = pca.transform(X_test_scaled)
    explained_variance_ratio = pca.explained_variance_ratio_

    return X_train_pca, X_test_pca, explained_variance_ratio, cumulative_variance

def plot_variance_charts(explained_variance_ratio, cumulative_variance, max_components):
    # Truncate arrays to max_components to ensure matching lengths
    explained_variance_ratio = explained_variance_ratio[:max_components]
    cumulative_variance = cumulative_variance[:max_components]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Bar chart of individual variances
    pcs = [f'PC{i+1}' for i in range(len(explained_variance_ratio))]
    ax1.bar(pcs, explained_variance_ratio * 100)
    ax1.set_title('Variance Captured by Each Principal Component')
    ax1.set_xlabel('Principal Components')
    ax1.set_ylabel('Variance Explained (%)')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, axis='y')
    
    # Bar chart with cumulative variance
    ax2.bar(pcs, explained_variance_ratio * 100, label='Individual Variance')
    ax2.plot(pcs, cumulative_variance * 100, color='red', marker='o', linestyle='--', label='Cumulative Variance')
    ax2.set_title('Variance with Cumulative Variance Overlay')
    ax2.set_xlabel('Principal Components')
    ax2.set_ylabel('Variance Explained (%)')
    ax2.tick_params(axis='x', rotation=45)
    ax2.legend()
    ax2.grid(True, axis='y')
    
    plt.tight_layout()
    plt.show()

def apply_tsne(X_train_scaled, tsne_perplexity=30.0):
    
    tsne = TSNE(n_components=2, perplexity=tsne_perplexity, random_state=42, verbose=0)
    X_train_tsne = tsne.fit_transform(X_train_scaled)
    
    plt.figure(figsize=(10, 6))
    # scatter = plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], cmap='viridis', s=50, alpha=0.6)
    scatter = plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], cmap='tab10', s=10, alpha=0.6)
    plt.colorbar(label='Target Value')
    plt.title(f't-SNE of Training Data (Perplexity={tsne_perplexity})')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.grid(True)
    plt.show()
    
    return X_train_tsne

In [ ]:
X_scaled_flat, _ = apply_standard_scaler(X_train=flattened)

print(f"Scaled data mean: {X_scaled_flat.mean():.2f}") 
print(f"Scaled data std: {X_scaled_flat.std():.2f}") 


In [ ]:
X_train_tsne = apply_tsne(X_scaled_flat)
# X_train_pca, X_test_pca, explained_variance_ratio, cumulative_variance = apply_pca(X_scaled_flat)